---
title: "Tutorial - Understanding the SBML DFs Pathway Format"
author: "Shackett"
date: "`r format(Sys.time(), '%d %B, %Y')`"
format:
  html:
    embed-resources: true
    code-fold: true
    code-summary: "Show the code"
    toc: true
    theme: minty
    highlight: tango
    code-block-bg: true
    code-block-border-left: "#5BB867"
jupyter: napistu_tutorials
---

In [ ]:
#| label: config
import logging
logger = logging.getLogger()
logger.setLevel("INFO")

import os
import pandas as pd
from napistu import utils

import tutorial_utils

config = tutorial_utils.CprConfig("config.yaml", "understanding_sbml_dfs")

# Overview

This tutorial aims to cover:
- What is the `SBML_dfs` format?
- How are `SBML_dfs` created?
- How can they be modified?

# Basic Structure

The [cpr wiki: sbml_dfs](https://github.com/calico/Open-CPR/wiki/SBML-DFs) provides a detailed overview of the `SBML_dfs` data structure. Here, we'll complement that description by showing the attributes of an actual pathway. To get up-and-running quickly we'll just load an existing `SBML_dfs` object constructed from merging multiple metabolic pathways into a consensus pathway.

In [ ]:
#| label: load_data
sbml_dfs = utils.load_pickle(config.artifacts["sbml_dfs"])

## Compartments

In [ ]:
#| label: compartments
utils.style_df(sbml_dfs.compartments)

## Chemical Species (genes, metabolites, complexes, drugs)

In [ ]:
#| label: species
utils.style_df(sbml_dfs.species.head())

## Compartmentalized Species (species in a compartment)

In [ ]:
#| label: cspecies
utils.style_df(sbml_dfs.compartmentalized_species.head())

## Reactions

In [ ]:
#| label: reactions
utils.style_df(sbml_dfs.reactions.head())

## Reactions

In [ ]:
#| label: reaction_species
utils.style_df(sbml_dfs.reaction_species.head())

## Optional attributes (`species_data` and `reactions_data`)

We can store additional data with species and reactions allowing us to connect molecular data to our graph. These tables `species_data` and `reactions_data` are discussed in the **adding_data_to_graphs** tutorial.

# Identifiers and Sources

Many of the core tables in `SBML_dfs` contain a column to track systematic identifiers and/or the information source. For example, `s_Identifiers` tracks a molecule's systematic identifiers while `s_Source` keeps track of the pathways including the molecules (this becomes relevant once we start merging network models; see the **merging_networks_as_a_consensus** tutorial).

## Identifiers

Identifiers are described in [cpr wiki: identifiers](https://github.com/calico/Open-CPR/wiki/SBML-DFs#identifiers) but here is an actual Identifiers object.

In [ ]:
#| label: identifiers_a
an_identifier = sbml_dfs.species["s_Identifiers"][0]
type(an_identifier)

At its core `Identifiers` objects are list of dictionaries

In [ ]:
#| label: identifiers_b
an_identifier.ids

Or, reformatted:

In [ ]:
#| label: identifiers_c
utils.style_df(pd.DataFrame(an_identifier.ids))

## Sources

While `Identifiers` track the systematic identifiers associated with entities `Source` objects track the pathway representation information came from. For a single-source `SBML_dfs` object its source's will be empty. For consensus model derived by combining multiple pathways, the `Source` object will track the upstream pathways. See [cpr wiki: sources](https://github.com/calico/Open-CPR/wiki/SBML-DFs#sources) for more details. Here, we can look more closely at a representative `Source` object to better understand its structure.

In [ ]:
#| label: source_a
a_source = sbml_dfs.species["s_Source"][0]
type(a_source)

Source objects core attribute is their `source` pd.DataFrame. Since we are loading an `SBML_dfs` object which was constructed by merging multiple upstream `SBML_dfs` (see dev/create_test_pathway.qmd to see how this was done) the source table tracks which upstream pathway(s) entities came from:

In [ ]:
#| label: source_b
utils.style_df(a_source.source)

When, an `SBML_dfs` model from a single-source is initialized its `Source.source` attributes are generally None. So the presense of source information indicates that the model has been created using the consensus module. This is module is described in the **merging_networks_as_a_consensus** tutorial.

# Creating `SBML_dfs`

## From a `.sbml` file

`SBML_dfs` pathway models can be created in three ways (from a .sbml file, using an edgelist format, and from raw tables). The latter two modes of creating sbml_dfs are shown later in this document but first we'll demonstrate how to create sbml_dfs from .sbml files because it is straight-forward and the sbml convention is arguably the most expressive format for accurately describing molecular mechanisms: [cpr wiki: sbml](https://github.com/calico/Open-CPR/wiki/SBML).

Here is how we can load and format the Reactome Glucose Metabolism pathway as an `SBML_dfs` object.

In [ ]:
#| label: sbml_dfs_from_sbml

from napistu import sbml_dfs_core
from napistu.ingestion import sbml
from napistu import utils

EXAMPLE_MODEL = "reactome_glucose_metabolism.sbml"
test_data = tutorial_utils.locate_test_data()

#| label: create_sbml_dfs
model_path = os.path.join(test_data, EXAMPLE_MODEL)
sbml_model = sbml.SBML(model_path)
sbml_dfs = sbml_dfs_core.SBML_dfs(sbml_model)

## From the underlying tables

In [ ]:
#| label: sbml_dfs_from_tables

sbml_tbl_dict = {
    "compartments": sbml_dfs.compartments,
    "species": sbml_dfs.species,
    "compartmentalized_species": sbml_dfs.compartmentalized_species,
    "reactions": sbml_dfs.reactions,
    "reaction_species": sbml_dfs.reaction_species,
}

sbml_dfs_core.SBML_dfs(sbml_tbl_dict).species

## From an edgelist

**TO DO**


# Modifying `sbml_dfs`

## Dropping cofactors

While cofactors such as ATP and NADH are biologically important they can generate confusion when we translate reactions into networks. For example, a dehydration reaction (X - H20) and a hydration reaction (Y + H20) would not really suggest regulation of Y by X via water. To deal with this issue, for many applications its helpful to remove cofactors.

In [ ]:
#| label: drop_cofactors
from napistu.modify import pathwayannot

filtered_pathway = pathwayannot.drop_cofactors(sbml_dfs)

## Uncompartmentalizing

Compartments are necessary to represent many mechanisms involving transport reactions such as creating a proton transport to fuel the electron transport chain. But, they add complexity which may be unecessary for some applications. By merging compartments we can create an uncompartmentalized model.

In [ ]:
#| label: uncompartmentalize
from napistu.modify import uncompartmentalize

one_compartment_sbml_dfs = uncompartmentalize.uncompartmentalize_sbml_dfs(sbml_dfs)

## Gap-filling

**TO DO**